ナイーブベイズ分類器による割り当て

In [5]:
import pandas as pd
import os

def read_fasta(file_path):
    """FASTAファイルを読み込む関数"""
    ids = []
    sequences = []
    with open(file_path, 'r') as f:
        while True:
            id_line = f.readline().strip()
            seq_line = f.readline().strip()
            if not id_line:  # EOF
                break
            ids.append(id_line[1:])  # '>'を除去
            sequences.append(seq_line)
    return pd.DataFrame({'#OTU ID': ids, 'sequence': sequences})



taxonomy_path = os.path.join('.', 'taxonomy.tsv')
feature_table_path = os.path.join('.', 'feature-table.tsv')
fasta_path = os.path.join('.', 'dna-sequences.fasta')
file_path = os.path.join('.', 'ASV0.xlsx')

# tsvファイルとfastaファイルを読み込みます
taxonomy_df = pd.read_csv(taxonomy_path, sep='\t', comment='#')

# tsvファイルの2行目（0-indexedなので1を指定）を読み込み、カラム名として使用します
columns = pd.read_csv(feature_table_path, sep='\t', skiprows=1, nrows=0).columns.tolist()
# skiprowsを使って最初の2行をスキップし、namesを使ってカラム名を指定します
feature_table_df = pd.read_csv(feature_table_path, sep='\t', skiprows=2, names=columns)

fasta_df = read_fasta(fasta_path)

# Feature ID でデータフレームをマージします
merged_df = pd.merge(taxonomy_df, feature_table_df, left_on='Feature ID', right_on='#OTU ID')
merged_df = pd.merge(merged_df, fasta_df, on='#OTU ID')

# Feature ID を削除し、OTU ID を ASV にリネーム
merged_df = merged_df.drop(columns='Feature ID')
merged_df = merged_df.rename(columns={'#OTU ID': 'ASV_ID', 'sequence': 'Representative sequence'})

# Taxonを';'でスプリットし、Order, Family, Genus, Speciesを抽出
taxon_splitted = merged_df['Taxon'].str.split(';', expand=True)

merged_df['Order'] = taxon_splitted[3].str.split('__', expand=True)[1]
merged_df['Family'] = taxon_splitted[4].str.split('__', expand=True)[1]
merged_df['Genus'] = taxon_splitted[5].str.split('__', expand=True)[1]
merged_df['Species'] = taxon_splitted[6].str.split('__', expand=True)[1]
merged_df['Species'] = merged_df['Species'].replace('NA', '')

merged_df = merged_df.drop(columns=['Taxon'])

# Speciesカラムの学名を２分法に統一する関数
def unify_species_name(name):
    if name is None:
        return None
    parts = name.split()
    if len(parts) > 2:
        return ' '.join(parts[:2])
    return name


# Speciesカラムの値を更新
merged_df['Species'] = merged_df['Species'].apply(unify_species_name)





In [6]:
# Blast検索の５つの結果をマージする
import pandas as pd
import os

df = pd.read_csv('blast_results.tsv', sep='\t', header=None)

df = df.iloc[:,[0,2,12,14]]

df.columns = ['query_id', 'science_name', 'blast_score', 'c_rate']

grouped = df.groupby('query_id').agg(list)

grouped.reset_index(inplace=True)



# 新しいカラム名を生成してデータを展開
for col in ['science_name', 'blast_score', 'c_rate']:
    for i in range(5):
        grouped[f'{col}_{i+1}'] = grouped[col].apply(lambda x: x[i] if i < len(x) else None)
        
grouped.drop(['science_name', 'blast_score', 'c_rate'], axis=1, inplace=True)



new_order = ['query_id']
for i in range(1,6):
    new_order.extend([f'science_name_{i}',f'c_rate_{i}',f'blast_score_{i}'])
grouped = grouped[new_order]


# ASV_IDとqseqidをキーカラムとして2つのデータフレームを連結
merged_df = pd.merge(merged_df, grouped, left_on='ASV_ID', right_on='query_id', how='left')

merged_df.drop(columns='query_id', inplace=True)

merged_df.loc[merged_df['Species'] != '','Species'] = merged_df['Genus'] + ' ' + merged_df['Species']




In [7]:
#blast検索の結果から最も一致率等が高い生物種を抜き出し、Qimme2の結果と比較して採用する分類結果を格納する
#閾値を調整するときは本セクションの15列目と35列目を調整する
#途中経過ファイルを出力する
import numpy as np

def select_best_match(row):
    best_match_name = None  # 正しく初期化
    highest_c_rate = 0
    highest_blast_score = 0
    
    for i in range(1, 5):
        c_rate = row[f'c_rate_{i}']  # インデックスを i に修正
        blast_score = row[f'blast_score_{i}']  # インデックスを i に修正
        
        if c_rate >= 98.7 and (c_rate > highest_c_rate or (c_rate == highest_c_rate and blast_score > highest_blast_score)):
            highest_c_rate = c_rate
            highest_blast_score = blast_score
            best_match_name = row[f'science_name_{i}']
    
    return best_match_name, highest_c_rate, highest_blast_score

# DataFrameの適用部分は変更なし
merged_df[['best_match_name', 'best_match_c_rate', 'best_match_blast_score']] = merged_df.apply(
    lambda row: pd.Series(select_best_match(row)),
    axis=1
)

def compare_classifier(row):
    if not pd.isna(row['best_match_name']):
        if row['best_match_name'] != row['Species']:
            return row['best_match_name']
        else:
            return row['Species']
    else:
        if row['Confidence'] >= 0.8:
            return row['Species']
        else:
            return None

merged_df['Species_new'] = merged_df.apply(compare_classifier, axis=1)  

merged_df.to_excel('temp_data.xlsx', index=False)

In [8]:
import pandas as pd

# Species_infoファイルを読み込む
species_info_path = 'Species_info'
species_df = pd.read_csv(species_info_path, header=None, names=['classification'])

# 分類情報を個別のカラムに分割
classification_columns = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
species_df[classification_columns] = species_df['classification'].str.split('; ', expand=True)

# 不要な接頭辞を削除
for col in classification_columns:
    species_df[col] = species_df[col].str.replace(r'^[a-z]__','', regex=True)

# 不要なカラムを削除
species_df.drop('classification', axis=1, inplace=True)

species_df['species'] = species_df['genus'] + ' ' + species_df['species']
species_df = species_df.drop_duplicates(subset='species')


# merged_dfのSpeciesカラムからGenusとSpeciesを抽出
merged_df['Species_tmp'] = merged_df['Species'].str.split().str[1]
merged_df['Genus_tmp'] = merged_df['Species'].str.split().str[0]
merged_df['Species_tmp2'] = merged_df['Species_new'].str.split().str[1]
merged_df['Genus_tmp2'] = merged_df['Species_new'].str.split().str[0]


# ①〜⑤の操作を実行
for idx, row in merged_df.iterrows():
    genus_tmp2 = row['Genus_tmp2']
    genus_tmp = row['Genus_tmp']
    
    # ②特になにもしません
    if genus_tmp2 == genus_tmp:
        continue
    
    # ③条件に当てはまる場合の操作
    if pd.isna(genus_tmp2) and pd.notna(genus_tmp):
        species = row['Species_new']
        species_match = species_df[species_df['species'] == species]
        if not species_match.empty:
            merged_df.at[idx, 'Order'] = species_match['order'].values[0]
            merged_df.at[idx, 'Family'] = species_match['family'].values[0]
            merged_df.at[idx, 'Genus'] = species_match['genus'].values[0]
    elif pd.isna(genus_tmp) and pd.notna(genus_tmp2):
        species = row['Species_new']
        species_match = species_df[species_df['species'] == species]
        if not species_match.empty:
            merged_df.at[idx, 'Order'] = species_match['order'].values[0]
            merged_df.at[idx, 'Family'] = species_match['family'].values[0]
            merged_df.at[idx, 'Genus'] = species_match['genus'].values[0]
    elif pd.notna(genus_tmp2) and pd.notna(genus_tmp) and genus_tmp2 != genus_tmp:
        species = row['Species_new']
        species_match = species_df[species_df['species'] == species]
        if not species_match.empty:
            merged_df.at[idx, 'Order'] = species_match['order'].values[0]
            merged_df.at[idx, 'Family'] = species_match['family'].values[0]
            merged_df.at[idx, 'Genus'] = species_match['genus'].values[0]


# 不要なカラムを削除
merged_df.drop([
    'Confidence',
    'Species',
    'science_name_1',
    'c_rate_1',
    'blast_score_1',
    'science_name_2',
    'c_rate_2',
    'blast_score_2',
    'science_name_3',
    'c_rate_3',
    'blast_score_3',
    'science_name_4',
    'c_rate_4',
    'blast_score_4',
    'science_name_5',
    'c_rate_5',
    'blast_score_5',
    'best_match_name',
    'best_match_c_rate',
    'best_match_blast_score',
    'Species_tmp',
    'Genus_tmp',
    'Species_tmp2',
    'Genus_tmp2'
], axis=1, inplace=True)

# Species_newカラムをSpeciesにリネーム
new_df = merged_df.rename(columns={'Species_new': 'Species'})

# Excelファイルとして出力
new_df.to_excel('output.xlsx', index=False)
